In [1]:
from aimsgb import GrainBoundary, Grain
import numpy as np

In [2]:
def read_poscar(filename):
    with open(filename, 'r') as f:
        lines = f.readlines()

    # Read the header
    system_name = lines[0].strip()

    # Read the scaling factor
    scale = float(lines[1].strip())

    # Read the lattice vectors
    lattice_vectors = []
    for i in range(2, 5):
        lattice_vectors.append([float(x) for x in lines[i].split()])
    lattice_vectors = np.array(lattice_vectors) * scale

    # Read the element symbols and counts
    elements = lines[5].split()
    counts = [int(x) for x in lines[6].split()]

    # Read the coordinate style
    coord_style = lines[7].strip().lower()

    # Read the atomic coordinates
    coords = []
    symbols = []
    start = 8
    for i, count in enumerate(counts):
        end = start + count
        symbols.extend([elements[i]] * count)
        for j in range(start, end):
            coords.append([float(x) for x in lines[j].split()[:3]])
        start = end

    # If the coordinate style is fractional, convert to Cartesian
    if coord_style == 'direct' or coord_style == 'cartesian':
        pass
    elif coord_style == 'fractional':
        coords = np.dot(coords, lattice_vectors)
    else:
        raise ValueError('Unsupported coordinate style: {}'.format(coord_style))

    return system_name, lattice_vectors, symbols, coords

def write_cel(filename, system_name, lattice_vectors, symbols, coords):
    with open(filename, 'w') as f:
        f.write('Comment string\n')
        f.write('0 {:.4f} {:.4f} {:.4f} 90.0000 90.0000 90.0000\n'.format(
            lattice_vectors[0][0]/10, lattice_vectors[1][1]/10, lattice_vectors[2][2]/10))
        for symbol, coord in zip(symbols, coords):
            f.write('{} {:.6f} {:.6f} {:.6f} 1.000000 0.005100 0.000000 0.000000 0.000000\n'.format(
                symbol, coord[0], coord[1], coord[2]))
        f.write('*\n')

In [19]:
s_input = Grain.from_mp_id("mp-2998")

gb = GrainBoundary([0, 0, 1], 5, [0, 0, 1], s_input, uc_a=5, uc_b=5)
structure = Grain.stack_grains(gb.grain_a, gb.grain_b, direction=gb.direction)

filename="BTO_[001]5[001]_uc5x_POSCAR"
structure.to(filename=filename)

poscar_file = filename
replaced_filename = filename.replace("_POSCAR", "")
cel_file = f"{replaced_filename}.cel"
xyz_file = f"{replaced_filename}.xyz"

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

In [20]:
system_name, lattice_vectors, symbols, coords = read_poscar(poscar_file)

In [21]:
lattice_vectors
lattice_vectors_transpose = np.transpose(lattice_vectors)
lattice_vectors_transpose
# new_lattice_vectors = np.matmul(lattice_vectors_transpose, [0, 0, 1])
# new_lattice_vectors

array([[-6.33670105e+00, -6.33670105e+00,  0.00000000e+00],
       [-6.33670105e+00,  6.33670105e+00,  0.00000000e+00],
       [-8.00000000e-16,  0.00000000e+00, -4.00768164e+01]])

In [22]:
coords[0]
np.matmul(lattice_vectors_transpose, coords[0])
coords_transformed = []
for i in range(len(coords)):
    coords_transformed.append(np.matmul(lattice_vectors_transpose, coords[i]))
coords_transformed


[array([ -5.06936084,  -3.80202063, -38.07297554]),
 array([ -5.06936084,  -3.80202063, -34.0652939 ]),
 array([ -5.06936084,  -3.80202063, -30.05761227]),
 array([ -5.06936084,  -3.80202063, -26.04993063]),
 array([ -5.06936084,  -3.80202063, -22.042249  ]),
 array([ -7.60404126,   3.80202063, -38.07297554]),
 array([ -7.60404126,   3.80202063, -34.0652939 ]),
 array([ -7.60404126,   3.80202063, -30.05761227]),
 array([ -7.60404126,   3.80202063, -26.04993063]),
 array([ -7.60404126,   3.80202063, -22.042249  ]),
 array([ -2.53468042,   1.26734021, -38.07297554]),
 array([ -2.53468042,   1.26734021, -34.0652939 ]),
 array([ -2.53468042,   1.26734021, -30.05761227]),
 array([ -2.53468042,   1.26734021, -26.04993063]),
 array([ -2.53468042,   1.26734021, -22.042249  ]),
 array([-10.13872168,  -1.26734021, -38.07297554]),
 array([-10.13872168,  -1.26734021, -34.0652939 ]),
 array([-10.13872168,  -1.26734021, -30.05761227]),
 array([-10.13872168,  -1.26734021, -26.04993063]),
 array([-10.

In [8]:
coords_transformed[0][0]

-5.06936084221635

In [17]:
def write_xyz(filename, symbols, coords_transformed):
    with open(filename, 'w') as f:
        f.write('{}\n'.format(
            len(coords_transformed)))
        f.write('comment string\n')
        for symbol, coord in zip(symbols, coords_transformed):
            f.write('{} {:.6f} {:.6f} {:.6f}\n'.format(
                symbol, coord[0], coord[1], coord[2]))

In [18]:
write_xyz(xyz_file, symbols, coords_transformed)